### Export Table A

Table A (~8000 rows  x  at least 6 columns  including Heat, wildfire, wildfire smoke >0, wildfire smoke > 5, population (2020), FIPS (11 digits) ) each entry is # number of exposure days all time  
 
2020 population data: From Esri 

In [45]:
import numpy as np
import pandas as pd

In [46]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [47]:
df = pd.read_parquet("outputs/hotspots_per_fips_rolling.parquet")

In [48]:
df = df[
    [
        "wfday",
        "GEOID",
        "smoke_pm",
        "heatday",
        "coldday",
        "polluted",
        "smoke_pm_non_zero",
        "smoke_pm_gt_five",
        "hw",
        "hp",
        "hwp",
        "hwps",
        "hs",
        "hws",
        "ws",
        "hs5",
        "hws5",
        "ws5",
    ]
]

df = df.groupby("GEOID").sum()

df = df.reset_index()

pop = pd.read_csv(
    "data/esri/crosswalk_ana/FIPSMapping_gdb_censustract_2020_ca.csv.csv",
    usecols=["FIPS", "POPULATION_2020", "POP20_SQMI", "SQMI"],
)

df = df.merge(pop, left_on="GEOID", right_on="FIPS")

df = df[
    [
        "GEOID",
        "heatday",
        "wfday",
        "coldday",
        "polluted",
        "smoke_pm_non_zero",
        "smoke_pm_gt_five",
        "hw",
        "hp",
        "hwp",
        "hwps",
        "hs",
        "hws",
        "ws",
        "hs5",
        "hws5",
        "ws5",
        "POPULATION_2020",
        "POP20_SQMI",
        "SQMI",
    ]
]

df.to_csv("table_a.csv", index=False)

In [49]:
df.head()

,GEOID,heatday,wfday,coldday,polluted,smoke_pm_non_zero,smoke_pm_gt_five,hw,hp,hwp,hwps,hs,hws,ws,hs5,hws5,ws5,POPULATION_2020,POP20_SQMI,SQMI
0,6001400100,289,0,228,34,433,134,0,7,0,756,61,0,0,25,0,0,3038,1133.6,2.68
1,6001400200,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,2001,8700.0,0.23
2,6001400300,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,5504,12800.0,0.43
3,6001400400,300,0,242,48,410,136,0,8,0,758,50,0,0,20,0,0,4112,14685.7,0.28
4,6001400500,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,3644,15843.5,0.23


## Export Table B: Add demographics

Data Request: Table B = Table A + columns of Index,  income, ethnicity (, % high school, Age group, whether owning a car, …  

Visualization (Ana)  
Distribution of sociodemographic variables at the census tract level in California.  
See Supplementary Figure 4:in https://iopscience.iop.org/article/10.1088/2752-5309/acc4e1/meta#erhacc4e1bib11 
Compare ours to their plotsto verify the code is correct 
 
Bivariate plots Age groups, Index, Income,  
 
  Visual 1: Bivariate plots of coexposure frequency and Social vulnerability Index  
  Visual 2: Bivariate plots of coexposure frequency and Income  
  Visual 3: Bivariate plots of coexposure frequency and Age  
  Visual 4: Average co-exposure frequency among black, white, Hispanic and the rest  
                    Reference: https://www.nature.com/articles/s41586-021-04190-y 

In [50]:
cols = [
    "FIPS",
    "E_TOTPOP",
    "M_TOTPOP",
    "E_HU",
    "M_HU",
    "E_HH",
    "M_HH",
    "EP_POV150",
    "MP_POV150",
    "EP_UNEMP",
    "MP_UNEMP",
    "EP_HBURD",
    "MP_HBURD",
    "EP_NOHSDP",
    "MP_NOHSDP",
    "EP_UNINSUR",
    "MP_UNINSUR",
    "EP_AGE65",
    "MP_AGE65",
    "EP_AGE17",
    "MP_AGE17",
    "EP_DISABL",
    "MP_DISABL",
    "EP_SNGPNT",
    "MP_SNGPNT",
    "EP_LIMENG",
    "MP_LIMENG",
    "EP_MINRTY",
    "MP_MINRTY",
    "EP_NOVEH",
    "MP_NOVEH",
    "RPL_THEME1",
    "RPL_THEME2",
    "RPL_THEME3",
    "RPL_THEME4",
    "RPL_THEMES",
]

In [51]:
svi = pd.read_csv("data/California.csv", usecols=cols, dtype={"FIPS": str})

In [52]:
df.head()

,GEOID,heatday,wfday,coldday,polluted,smoke_pm_non_zero,smoke_pm_gt_five,hw,hp,hwp,hwps,hs,hws,ws,hs5,hws5,ws5,POPULATION_2020,POP20_SQMI,SQMI
0,6001400100,289,0,228,34,433,134,0,7,0,756,61,0,0,25,0,0,3038,1133.6,2.68
1,6001400200,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,2001,8700.0,0.23
2,6001400300,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,5504,12800.0,0.43
3,6001400400,300,0,242,48,410,136,0,8,0,758,50,0,0,20,0,0,4112,14685.7,0.28
4,6001400500,300,0,242,54,410,136,0,8,0,764,50,0,0,20,0,0,3644,15843.5,0.23


In [53]:
svi.FIPS = svi.FIPS.astype(int)

In [54]:
dfsvi = df.merge(svi, left_on="GEOID", right_on="FIPS")

In [55]:
len(dfsvi)

8673

In [32]:
dfsvi.to_csv("table_b.csv", index=False)

## Time trend analysis

Objective 3: time trend analysis   
 
Data Request: Table C = Table B + Year column (8000 x 15 rows) 
Statistics: (Kate)  
Test whether there is a year trend 
Fit regression models of co-exposure days ~ year  
Test whether there is the disparity varies over years 
Fit regression models of co-exposure days ~ \alpha1 + \beta1*year + \beta2*social economic status + \beta3*year*social economic status  

(Ana) Plots of social economic status 
for each of the exposure and the co-exposurve of three over time (4 sets of figures)   
See Supplementary Figure 1: Five measures of census tract-level wildfire PM2.5 exposure by year from 2006–2020 in https://www.researchsquare.com/article/rs-2866201/v1 

In [15]:
df = pd.read_parquet("outputs/hotspots_per_fips_rolling.parquet")

In [16]:
trends_df = (
    df.groupby("GEOID")[
        [
            "time",
            "ws",
            "hw",
            "hp",
            "hs",
            "hwp",
            "hws",
            "hwps",
            "wfday",
            "hs5",
            "hws5",
            "ws5",
            "heatday",
            "coldday",
            "polluted",
            "smoke_pm_non_zero",
            "smoke_pm_gt_five",
        ]
    ]
    .resample("Y", on="time")
    .sum()
)

In [18]:
trends_df = trends_df.reset_index()

In [19]:
trends_df["year"] = trends_df["time"].dt.year

In [21]:
trends_df = trends_df[
    [
        "GEOID",
        "year",
        "ws",
        "hw",
        "hp",
        "hs",
        "hwp",
        "hws",
        "wfday",
        "heatday",
        "polluted",
        "smoke_pm_non_zero",
    ]
]

In [30]:
varnames = [
    # "ws", # first
    "hw",
    "hp",
    "hs",
    "hwp",
    "hws",
    "wfday",
    "heatday",
    "polluted",
    "smoke_pm_non_zero",
]

In [27]:
def format_table(df, variable):
    pivoted = trends_df.pivot_table(
        index="GEOID", columns="year", values=variable
    ).add_prefix(variable + "_")
    return pivoted

In [39]:
result = format_table(trends_df, "ws")

In [40]:
for varname in varnames:
    temp = format_table(trends_df, varname)
    result = pd.concat([result, temp], axis=1)

In [37]:
result.reset_index(inplace=True)

In [60]:
len(result)

8673

In [59]:
len(dfsvi)

8673

In [64]:
result = dfsvi.groupby("GEOID").first().join(result)

In [65]:
result.isna().sum()

heatday                   0
wfday                     0
coldday                   0
polluted                  0
smoke_pm_non_zero         0
                         ..
smoke_pm_non_zero_2016    0
smoke_pm_non_zero_2017    0
smoke_pm_non_zero_2018    0
smoke_pm_non_zero_2019    0
smoke_pm_non_zero_2020    0
Length: 205, dtype: int64

In [66]:
result.to_csv("table_c.csv")